# prototype-sequence-engineer 

Feature engineering methods for sequence data

NCH 2022

## Sub-task: window-based feature extraction

In [320]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from numpy.lib.stride_tricks import sliding_window_view

## Functions

In [321]:
def to_window(x, shape, stride):
    value = sliding_window_view(x,shape)[::stride,:]
    return value


def to_frame(x, columns=None):
    return pd.DataFrame(x, columns=columns)
    
    
def to_series(x):
    return pd.Series(x.tolist())


def window_info(ind, shape, stride):
    value = sliding_window_view(ind,shape)[::stride,:]
    n_element = len(ind)
    n_window = np.floor((n_element-shape)/stride)+1
    value = {
        "index": value,
        "n_element": int(n_element),
        "n_window": int(n_window)
        }
    return value


def window_stat(x):
    #see Metric enumeration 
    metrics = np.empty( (x.shape[0], 15) )
    metrics[:,0]  = np.nanmean(x, axis=1)
    metrics[:,1]  = np.nanstd(x, axis=1)
    metrics[:,2]  = np.nanmin(x, axis=1) 
    metrics[:,3]  = np.nanmax(x, axis=1)
    metrics[:,4]  = np.nanmedian(x, axis=1)
    metrics[:,5]  = np.nanmedian(np.absolute(x - np.nanmedian(x, axis=1).reshape(-1,1)), axis=1)
    metrics[:,6]  = np.nanmean(np.absolute(x - np.nanmean(x, axis=1).reshape(-1,1)), axis=1)
    metrics[:,7]  = np.nanmax(x, axis=1) - np.nanmin(x, axis=1)
    metrics[:,8]  = np.percentile(x, 75, axis=1) - np.percentile(x, 25, axis=1)
    metrics[:,9]  = np.nansum(x > 0, axis=1)
    metrics[:,10] = np.nansum(x < 0, axis=1)
    metrics[:,11] = np.nansum(x > np.nanmean(x, axis=1).reshape(-1,1), axis=1)
    metrics[:,12] = stats.skew(x, axis=1)
    metrics[:,13] = stats.kurtosis(x, axis=1)
    metrics[:,14] = np.nansum((x**2)/100, axis=1)
    return metrics


def get_metric_name(column_name):
    return [column_name + "_" + metric for metric in Metrics.list()]


def window_engineer(df, features, by=None, shape=None, stride=1):

    if isinstance(features, str):
        features = [features]
        
    df_eng  = pd.DataFrame()
    if not by:
       df_eng = _window_engineer(df[features], shape, stride)
    else:
        grouped = df.groupby(by)
        for _, group in grouped:    
            grp_eng = _window_engineer(group[features], shape, stride)
            df_eng = pd.concat([df_eng, grp_eng], axis=0)
    return df_eng


def _window_engineer(df, *args):
    # couldn't land on a straightforward implementation using apply 
    # instead implementation loops over the cheap dimension [col] an vectorizes the expensive one [rows]

    info = window_info(df.index, *args)
    accumulated_names   = []
    accumulated_metrics = np.empty((info["n_window"], 0))
    for column in df:
        array = to_window(df[column], *args)
        metrics = window_stat( array )
        accumulated_names.extend( get_metric_name(column) )
        accumulated_metrics = np.c_[accumulated_metrics, metrics]
    df_eng = to_frame( accumulated_metrics, 
        columns = accumulated_names )
    return df_eng


from enum import Enum, unique 
class ExtendedEnum(Enum):
    @classmethod
    def list(cls):
        return list(map(lambda c: c.value, cls))

@unique 
class Metrics(ExtendedEnum):
    mean    = "mean" 
    std     = "std"
    min     = "min"
    max     = "max"
    median  = "median"
    mad     = "mad"
    aad     = "aad"
    range   = "range"
    iqr     = "iqr"
    pc      = "pc"
    nc      = "nc"
    vam     = "vam"
    skew    = "skew"
    kurt    = "kurt"
    energy  = "energy"



## Setup a synthetic dataset for prototype

Synthetic data for development. Using sequential values to make easier to debug. 

In [322]:
#numeric sequence w/ nan
col1    = np.arange(0,10, dtype=float)
col1[7] = np.nan

#numeric sequence w/o nan 
col2    = np.arange(0,10, dtype=float) + 10

#string sequence 
col3    =  np.random.choice(['rabbit', 'piglet'], len(col1))

In [323]:
data = pd.DataFrame(zip(col1, col2, col3), 
    columns = ["X", "Y", "Z"], 
    index = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"])
data

,X,Y,Z
A,0.0,10.0,piglet
B,1.0,11.0,piglet
C,2.0,12.0,piglet
D,3.0,13.0,rabbit
E,4.0,14.0,rabbit
F,5.0,15.0,rabbit
G,6.0,16.0,piglet
H,NaN,17.0,rabbit
I,8.0,18.0,rabbit
J,9.0,19.0,piglet


In [324]:
grps = data.groupby("Z")
grps.get_group("rabbit")

,X,Y,Z
D,3.0,13.0,rabbit
E,4.0,14.0,rabbit
F,5.0,15.0,rabbit
H,NaN,17.0,rabbit
I,8.0,18.0,rabbit


Specify the window design

In [325]:
shape  = 3
stride = 2

Preview the window design using the frame index, and return the shape metadata.

In [326]:
meta = window_info(data.index, shape, stride)
meta

{'index': array([['A', 'B', 'C'],
        ['C', 'D', 'E'],
        ['E', 'F', 'G'],
        ['G', 'H', 'I']], dtype=object),
 'n_element': 10,
 'n_window': 4}

Convert a feature column to windows using numpy `stride_tricks`. Control dimension and overlap via shape and stride. 

In [327]:
output = to_window(data["X"], shape, stride)
output

array([[ 0.,  1.,  2.],
       [ 2.,  3.,  4.],
       [ 4.,  5.,  6.],
       [ 6., nan,  8.]])

Vectorized feature extraction in numpy. The product is an `m instance x n feature` numpy array for each input (raw feature). See `Metrics.list()` for feature list enumeration. 

In [328]:
Metrics.list()

['mean',
 'std',
 'min',
 'max',
 'median',
 'mad',
 'aad',
 'range',
 'iqr',
 'pc',
 'nc',
 'vam',
 'skew',
 'kurt',
 'energy']

In [329]:
window_stat(output)

array([[ 1.        ,  0.81649658,  0.        ,  2.        ,  1.        ,
         1.        ,  0.66666667,  2.        ,  1.        ,  2.        ,
         0.        ,  1.        ,  0.        , -1.5       ,  0.05      ],
       [ 3.        ,  0.81649658,  2.        ,  4.        ,  3.        ,
         1.        ,  0.66666667,  2.        ,  1.        ,  3.        ,
         0.        ,  1.        ,  0.        , -1.5       ,  0.29      ],
       [ 5.        ,  0.81649658,  4.        ,  6.        ,  5.        ,
         1.        ,  0.66666667,  2.        ,  1.        ,  3.        ,
         0.        ,  1.        ,  0.        , -1.5       ,  0.77      ],
       [ 7.        ,  1.        ,  6.        ,  8.        ,  7.        ,
         1.        ,  1.        ,  2.        ,         nan,  2.        ,
         0.        ,  1.        ,         nan,         nan,  1.        ]])

Window based feature extraction w/o group support [helper component]

In [330]:
_window_engineer( data[["X","Y"]], shape, stride )


,X_mean,X_std,X_min,X_max,X_median,X_mad,X_aad,X_range,X_iqr,X_pc,...,Y_mad,Y_aad,Y_range,Y_iqr,Y_pc,Y_nc,Y_vam,Y_skew,Y_kurt,Y_energy
0,1.0,0.816497,0.0,2.0,1.0,1.0,0.666667,2.0,1.0,2.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,3.65
1,3.0,0.816497,2.0,4.0,3.0,1.0,0.666667,2.0,1.0,3.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,5.09
2,5.0,0.816497,4.0,6.0,5.0,1.0,0.666667,2.0,1.0,3.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,6.77
3,7.0,1.000000,6.0,8.0,7.0,1.0,1.000000,2.0,NaN,2.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,8.69


Window based feature extraction with or without specified grouping

In [331]:
window_engineer(data, ["X", "Y"], shape=shape, stride=stride)

,X_mean,X_std,X_min,X_max,X_median,X_mad,X_aad,X_range,X_iqr,X_pc,...,Y_mad,Y_aad,Y_range,Y_iqr,Y_pc,Y_nc,Y_vam,Y_skew,Y_kurt,Y_energy
0,1.0,0.816497,0.0,2.0,1.0,1.0,0.666667,2.0,1.0,2.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,3.65
1,3.0,0.816497,2.0,4.0,3.0,1.0,0.666667,2.0,1.0,3.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,5.09
2,5.0,0.816497,4.0,6.0,5.0,1.0,0.666667,2.0,1.0,3.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,6.77
3,7.0,1.000000,6.0,8.0,7.0,1.0,1.000000,2.0,NaN,2.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.0,-1.5,8.69


In [332]:
window_engineer(data, ["X", "Y"], by="Z" ,shape=shape, stride=stride)

,X_mean,X_std,X_min,X_max,X_median,X_mad,X_aad,X_range,X_iqr,X_pc,...,Y_mad,Y_aad,Y_range,Y_iqr,Y_pc,Y_nc,Y_vam,Y_skew,Y_kurt,Y_energy
0,1.000000,0.816497,0.0,2.0,1.0,1.0,0.666667,2.0,1.0,2.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.000000,-1.5,3.65
1,5.666667,2.867442,2.0,9.0,6.0,3.0,2.444444,7.0,3.5,3.0,...,3.0,2.444444,7.0,3.5,3.0,0.0,2.0,-0.172801,-1.5,7.61
0,4.000000,0.816497,3.0,5.0,4.0,1.0,0.666667,2.0,1.0,3.0,...,1.0,0.666667,2.0,1.0,3.0,0.0,1.0,0.000000,-1.5,5.90
1,6.500000,1.500000,5.0,8.0,6.5,1.5,1.500000,3.0,NaN,2.0,...,1.0,1.111111,3.0,1.5,3.0,0.0,2.0,-0.381802,-1.5,8.38



**resources**
+ [Feature engineering on time-series data](https://towardsdatascience.com/feature-engineering-on-time-series-data-transforming-signal-data-of-a-smartphone-accelerometer-for-72cbe34b8a60)


"unpanda" experiment

In [333]:
df = pd.DataFrame(
    pd.Series( output.tolist()),
    columns=["X"],
    )
df


,X
0,"[0.0, 1.0, 2.0]"
1,"[2.0, 3.0, 4.0]"
2,"[4.0, 5.0, 6.0]"
3,"[6.0, nan, 8.0]"
